# 🏴‍☠️ AN2DL25 Challenge 1 — Pirate Pain Classification

This notebook implements a full deep-learning pipeline for multivariate time-series classification of the Pirate Pain dataset. It is inspired by the Lecture 4 notebook (`Timeseries Classification (1).ipynb`) but adapted for the competition setting, including data preparation, model training (RNN/GRU/LSTM variants), evaluation, and test-time inference.



In [ ]:
# %%capture
# !pip install -q -r requirements.txt




In [ ]:
import os
import random
import math
import copy
from pathlib import Path
from typing import Tuple, Dict, Optional, List
from datetime import datetime
from itertools import product
import inspect

import numpy as np
import pandas as pd
from sklearn.metrics import (
    f1_score,
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support,
)
from sklearn.model_selection import StratifiedKFold, train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

try:
    from torch.amp import autocast, GradScaler
except ImportError:  # pragma: no cover
    from torch.cuda.amp import autocast, GradScaler


def set_seed(seed: int = 42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False



In [ ]:
SEED = 42

try:
    from google.colab import drive  # type: ignore
    IN_COLAB = True
except ImportError:  # pragma: no cover
    IN_COLAB = False

if IN_COLAB:
    drive.mount('/content/drive', force_remount=False)
    BASE_DIR = Path('/content/drive/MyDrive/[2025-2026]\ A2NDL/Challenge')
else:
    BASE_DIR = Path('/Users/md101ta/Desktop/Pirates')

DATA_DIR = (BASE_DIR / 'data').resolve()
OUTPUT_DIR = (BASE_DIR / 'outputs').resolve()
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# Reproducibility
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    DEVICE = torch.device('cuda')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    DEVICE = torch.device('cpu')

print(f'Running in Colab: {IN_COLAB}')
print(f'Device: {DEVICE}')
print(f'Data dir: {DATA_DIR}')
print(f'Output dir: {OUTPUT_DIR}')

_AUTocast_params = inspect.signature(autocast).parameters
_GRADSCALER_PARAMS = inspect.signature(GradScaler).parameters

def autocast_context():
    enabled = DEVICE.type == 'cuda'
    if 'device_type' in _AUTocast_params:
        return autocast(device_type=DEVICE.type, enabled=enabled)
    if 'device' in _AUTocast_params:
        return autocast(DEVICE.type, enabled=enabled)
    # fallback to legacy signature (enabled only)
    return autocast(enabled=enabled)


def create_grad_scaler():
    enabled = DEVICE.type == 'cuda'
    if 'device_type' in _GRADSCALER_PARAMS:
        return GradScaler(device_type=DEVICE.type, enabled=enabled)
    return GradScaler(enabled=enabled)


In [ ]:
LOG_DIR = (OUTPUT_DIR / 'logs').resolve()
CHECKPOINT_DIR = (OUTPUT_DIR / 'checkpoints').resolve()
LOG_DIR.mkdir(exist_ok=True, parents=True)
CHECKPOINT_DIR.mkdir(exist_ok=True, parents=True)



In [ ]:
from pathlib import Path
import pandas as pd
from typing import Tuple

# Percorso al dataset su Google Drive
DATA_DIR = Path('/content/drive/MyDrive/[2025-2026] AN2DL/Challenge')

def load_data(data_dir: Path) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    X_train = pd.read_csv(data_dir / 'pirate_pain_train.csv')
    y_train = pd.read_csv(data_dir / 'pirate_pain_train_labels.csv')
    X_test  = pd.read_csv(data_dir / 'pirate_pain_test.csv')
    return X_train, y_train, X_test

# Carica i dati
X_train_raw, y_train, X_test_raw = load_data(DATA_DIR)
print(X_train_raw.shape, y_train.shape, X_test_raw.shape)




In [ ]:
CATEGORICAL_COLUMNS = ['n_legs', 'n_hands', 'n_eyes']
CATEGORY_MAPPINGS: Dict[str, Dict[str, int]] = {}

for col in CATEGORICAL_COLUMNS:
    uniques = pd.concat([X_train_raw[col], X_test_raw[col]]).dropna().unique()
    mapping = {value: idx for idx, value in enumerate(sorted(uniques))}
    CATEGORY_MAPPINGS[col] = mapping
    X_train_raw[col] = X_train_raw[col].map(mapping).astype(np.int32)
    X_test_raw[col] = X_test_raw[col].map(mapping).astype(np.int32)

FEATURE_COLUMNS = [col for col in X_train_raw.columns if col not in ['sample_index', 'time']]
TIME_STEPS = X_train_raw['time'].nunique()
NUM_FEATURES = len(FEATURE_COLUMNS)
NUM_CLASSES = y_train['label'].nunique()
print(f'Time steps: {TIME_STEPS} | Features: {NUM_FEATURES} | Classes: {NUM_CLASSES}')
print('Category mappings:', CATEGORY_MAPPINGS)

y_train['label'].value_counts().plot(kind='bar', title='Class distribution')
plt.show()



In [ ]:
LABEL2IDX = {label: idx for idx, label in enumerate(sorted(y_train['label'].unique()))}
IDX2LABEL = {idx: label for label, idx in LABEL2IDX.items()}
print('Label mapping:', LABEL2IDX)


def pivot_timeseries(df: pd.DataFrame) -> np.ndarray:
    pivoted = df.set_index(['sample_index', 'time'])[FEATURE_COLUMNS].unstack(level=1)
    times = sorted(df['time'].unique())
    pivoted = pivoted.reindex(columns=pd.MultiIndex.from_product([FEATURE_COLUMNS, times]))
    values = pivoted.to_numpy().reshape(-1, NUM_FEATURES, len(times)).transpose(0, 2, 1)
    return values


X_train_np = pivot_timeseries(X_train_raw)
X_test_np = pivot_timeseries(X_test_raw)
y_train_idx = y_train.set_index('sample_index').loc[pd.unique(X_train_raw['sample_index'])]['label'].map(LABEL2IDX).to_numpy()

print(X_train_np.shape, y_train_idx.shape, X_test_np.shape)



In [ ]:
def compute_normalization_stats(data: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    # data shape: (N, T, F)
    mean = data.reshape(-1, NUM_FEATURES).mean(axis=0)
    std = data.reshape(-1, NUM_FEATURES).std(axis=0) + 1e-6
    return mean, std


def normalize(data: np.ndarray, mean: np.ndarray, std: np.ndarray) -> np.ndarray:
    return (data - mean) / std


feat_mean, feat_std = compute_normalization_stats(X_train_np)
X_train_np = normalize(X_train_np, feat_mean, feat_std)
X_test_np = normalize(X_test_np, feat_mean, feat_std)



In [ ]:
def make_dataloader_from_arrays(
    X: np.ndarray,
    y: Optional[np.ndarray],
    batch_size: int,
    shuffle: bool,
    seed: int = SEED,
    num_workers: int = 0,
    pin_memory: bool = False,
) -> DataLoader:
    dataset = TimeSeriesDataset(X, y)
    generator = torch.Generator().manual_seed(seed) if shuffle else None
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=False,
        num_workers=num_workers,
        pin_memory=pin_memory,
        generator=generator,
    )



In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data: np.ndarray, labels: Optional[np.ndarray] = None):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = None if labels is None else torch.tensor(labels, dtype=torch.long)

    def __len__(self) -> int:
        return self.data.shape[0]

    def __getitem__(self, idx: int):
        if self.labels is None:
            return self.data[idx]
        return self.data[idx], self.labels[idx]



In [ ]:
def create_dataloaders(
    X: np.ndarray,
    y: np.ndarray,
    valid_size: float = 0.1,
    batch_size: int = 64,
    seed: int = SEED,
    num_workers: int = 0,
    pin_memory: bool = False,
) -> Tuple[DataLoader, DataLoader, Tuple[np.ndarray, ...], Tuple[np.ndarray, np.ndarray]]:
    idx_all = np.arange(len(y))
    X_train, X_valid, y_train, y_valid, train_idx, valid_idx = train_test_split(
        X,
        y,
        idx_all,
        test_size=valid_size,
        random_state=seed,
        stratify=y,
    )

    train_loader = make_dataloader_from_arrays(
        X_train,
        y_train,
        batch_size=batch_size,
        shuffle=True,
        seed=seed,
        num_workers=num_workers,
        pin_memory=pin_memory,
    )
    valid_loader = make_dataloader_from_arrays(
        X_valid,
        y_valid,
        batch_size=batch_size,
        shuffle=False,
        seed=seed,
        num_workers=num_workers,
        pin_memory=pin_memory,
    )
    return train_loader, valid_loader, (X_train, y_train, X_valid, y_valid), (train_idx, valid_idx)


DEFAULT_BATCH_SIZE = 64
train_loader, valid_loader, (X_train_split, y_train_split, X_valid_split, y_valid_split), _ = create_dataloaders(
    X_train_np,
    y_train_idx,
    batch_size=DEFAULT_BATCH_SIZE,
)



In [ ]:
def label_smoothing_targets(num_classes: int, smoothing: float) -> torch.Tensor:
    assert 0.0 <= smoothing < 1.0
    return torch.full((num_classes,), smoothing / (num_classes - 1), dtype=torch.float32)


def build_criterion(config: Dict, class_counts: Optional[np.ndarray] = None) -> nn.Module:
    weight = None
    if config.get('use_class_weights', False) and class_counts is not None:
        total = class_counts.sum()
        weight = torch.tensor([total / max(c, 1) for c in class_counts], dtype=torch.float32)
    if config.get('use_focal_loss', False):
        alpha = weight if weight is not None else None
        return FocalLoss(alpha=alpha, gamma=config.get('focal_gamma', 2.0))
    smoothing = config.get('label_smoothing', 0.0)
    return nn.CrossEntropyLoss(
        weight=weight,
        label_smoothing=smoothing,
    )



In [ ]:
class RecurrentBackbone(nn.Module):
    def __init__(
        self,
        input_size: int,
        hidden_size: int = 128,
        num_layers: int = 2,
        dropout: float = 0.2,
        rnn_type: str = 'lstm',
        bidirectional: bool = True,
    ):
        super().__init__()
        rnn_cls = {
            'rnn': nn.RNN,
            'gru': nn.GRU,
            'lstm': nn.LSTM,
        }[rnn_type.lower()]
        self.rnn_type = rnn_type.lower()
        self.bidirectional = bidirectional
        self.rnn = rnn_cls(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True,
            bidirectional=bidirectional,
        )
        proj_input = hidden_size * (2 if bidirectional else 1)
        self.head = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(proj_input, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, NUM_CLASSES),
        )

    def forward(self, x):
        out, _ = self.rnn(x)
        # use last time-step hidden state
        last = out[:, -1, :]
        return self.head(last)



In [ ]:
def compute_classification_metrics(preds: np.ndarray, targets: np.ndarray) -> Dict[str, float]:
    accuracy = float((preds == targets).mean())
    precision, recall, f1, _ = precision_recall_fscore_support(
        targets,
        preds,
        average='macro',
        zero_division=0,
    )
    return {
        'accuracy': accuracy,
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
    }


def train_one_epoch(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    scaler: GradScaler,
    max_grad_norm: float = 5.0,
) -> Tuple[float, Dict[str, float]]:
    model.train()
    running_loss = 0.0
    preds_all, targets_all = [], []

    for inputs, targets in loader:
        inputs = inputs.to(DEVICE)
        targets = targets.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)

        with autocast_context():
            logits = model(inputs)
            loss = criterion(logits, targets)

        scaler.scale(loss).backward()
        if max_grad_norm is not None:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * inputs.size(0)
        preds_all.append(torch.argmax(logits.detach(), dim=1).cpu())
        targets_all.append(targets.cpu())

    preds_np = torch.cat(preds_all).numpy()
    targets_np = torch.cat(targets_all).numpy()
    metrics = compute_classification_metrics(preds_np, targets_np)
    avg_loss = running_loss / len(loader.dataset)
    return avg_loss, metrics


def evaluate_epoch(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
) -> Tuple[float, Dict[str, float], np.ndarray, np.ndarray]:
    model.eval()
    running_loss = 0.0
    preds_all, targets_all = [], []

    with torch.no_grad():
        for inputs, targets in loader:
            inputs = inputs.to(DEVICE)
            targets = targets.to(DEVICE)
            with autocast_context():
                logits = model(inputs)
                loss = criterion(logits, targets)

            running_loss += loss.item() * inputs.size(0)
            preds_all.append(torch.argmax(logits, dim=1).cpu())
            targets_all.append(targets.cpu())

    preds_np = torch.cat(preds_all).numpy()
    targets_np = torch.cat(targets_all).numpy()
    metrics = compute_classification_metrics(preds_np, targets_np)
    avg_loss = running_loss / len(loader.dataset)
    return avg_loss, metrics, preds_np, targets_np



In [ ]:
def fit_model(
    config: Dict,
    train_loader: DataLoader,
    valid_loader: DataLoader,
    run_name: str,
    criterion: Optional[nn.Module] = None,
    tensorboard: bool = True,
) -> Tuple[nn.Module, Dict[str, List[float]], Dict]:
    model = RecurrentBackbone(
        input_size=NUM_FEATURES,
        hidden_size=config['hidden_size'],
        num_layers=config['num_layers'],
        dropout=config['dropout'],
        rnn_type=config['rnn_type'],
        bidirectional=config.get('bidirectional', False),
    ).to(DEVICE)

    if criterion is None:
        criterion = build_criterion(config)
    if hasattr(criterion, 'to'):
        criterion = criterion.to(DEVICE)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config['lr'],
        weight_decay=config.get('weight_decay', 0.0),
    )

    scheduler = None
    if config.get('scheduler', 'plateau') == 'plateau':
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='max',
            factor=config.get('scheduler_factor', 0.5),
            patience=config.get('scheduler_patience', 3),
            threshold=config.get('scheduler_threshold', 1e-3),
            cooldown=config.get('scheduler_cooldown', 0),
        )

    scaler = create_grad_scaler()

    history: Dict[str, List[float]] = {
        'train_loss': [],
        'train_accuracy': [],
        'train_f1': [],
        'train_precision': [],
        'train_recall': [],
        'valid_loss': [],
        'valid_accuracy': [],
        'valid_f1': [],
        'valid_precision': [],
        'valid_recall': [],
        'valid_f1_ema': [],
        'lr': [],
    }

    run_log_dir = (LOG_DIR / run_name).resolve()
    writer = SummaryWriter(run_log_dir.as_posix()) if tensorboard else None

    metric_key = config.get('metric_for_best', 'macro_f1')
    patience = config.get('patience', 10)
    min_improvement = config.get('min_improvement', 0.0)
    warmup_epochs = config.get('warmup_epochs', 0)
    base_lr = config['lr']
    ema_alpha = config.get('ema_alpha', 0.1)

    best_ema = -np.inf
    best_metric_value = -np.inf
    ema_value = None
    patience_counter = 0
    checkpoint_path = (CHECKPOINT_DIR / f'{run_name}.pt').resolve()
    best_state: Optional[Dict] = None

    for epoch in range(1, config['epochs'] + 1):
        train_loss, train_metrics = train_one_epoch(
            model,
            train_loader,
            criterion,
            optimizer,
            scaler,
            max_grad_norm=config.get('max_grad_norm', 5.0),
        )
        valid_loss, valid_metrics, preds, targets = evaluate_epoch(
            model,
            valid_loader,
            criterion,
        )

        metric_value = valid_metrics['f1'] if metric_key in {'macro_f1', 'f1'} else valid_metrics.get('accuracy', 0.0)
        ema_value = metric_value if ema_value is None else (ema_alpha * metric_value + (1 - ema_alpha) * ema_value)

        if warmup_epochs > 0 and epoch <= warmup_epochs:
            lr_now = base_lr * (epoch / warmup_epochs)
            for group in optimizer.param_groups:
                group['lr'] = lr_now
        elif scheduler is not None:
            scheduler.step(metric_value)

        current_lr = optimizer.param_groups[0]['lr']

        history['train_loss'].append(train_loss)
        history['train_accuracy'].append(train_metrics['accuracy'])
        history['train_f1'].append(train_metrics['f1'])
        history['train_precision'].append(train_metrics['precision'])
        history['train_recall'].append(train_metrics['recall'])
        history['valid_loss'].append(valid_loss)
        history['valid_accuracy'].append(valid_metrics['accuracy'])
        history['valid_f1'].append(valid_metrics['f1'])
        history['valid_precision'].append(valid_metrics['precision'])
        history['valid_recall'].append(valid_metrics['recall'])
        history['valid_f1_ema'].append(ema_value)
        history['lr'].append(current_lr)

        if writer is not None:
            writer.add_scalar('Loss/train', train_loss, epoch)
            writer.add_scalar('Loss/valid', valid_loss, epoch)
            writer.add_scalar('F1/macro', valid_metrics['f1'], epoch)
            writer.add_scalar('F1/macro_ema', ema_value, epoch)
            writer.add_scalar('Accuracy/train', train_metrics['accuracy'], epoch)
            writer.add_scalar('Accuracy/valid', valid_metrics['accuracy'], epoch)
            writer.add_scalar('LearningRate', current_lr, epoch)
            try:
                rep = classification_report(targets, preds, output_dict=True, zero_division=0)
                for cls_name, stats in rep.items():
                    if isinstance(stats, dict) and 'f1-score' in stats:
                        writer.add_scalar(f'F1_class/{cls_name}', stats['f1-score'], epoch)
                cm = confusion_matrix(targets, preds)
                fig, ax = plt.subplots()
                ax.imshow(cm, interpolation='nearest', cmap='Blues')
                ax.set_title('Confusion Matrix')
                ax.set_xlabel('Predicted')
                ax.set_ylabel('True')
                for (i, j), val in np.ndenumerate(cm):
                    ax.text(j, i, int(val), ha='center', va='center')
                writer.add_figure('ConfusionMatrix', fig, epoch)
                plt.close(fig)
            except Exception:
                pass

        msg = (
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_metrics['accuracy']:.3f} f1={train_metrics['f1']:.3f} | "
            f"valid_loss={valid_loss:.4f} acc={valid_metrics['accuracy']:.3f} f1={valid_metrics['f1']:.3f} | "
            f"lr={current_lr:.2e}"
        )
        print(msg)

        improved = ema_value > best_ema + min_improvement
        if improved:
            best_ema = ema_value
            best_metric_value = metric_value
            patience_counter = 0
            best_state = {
                'epoch': epoch,
                'model_state': {k: v.cpu() for k, v in model.state_dict().items()},
                'optimizer_state': optimizer.state_dict(),
                'metrics': valid_metrics,
                'train_metrics': train_metrics,
                'preds': preds,
                'targets': targets,
                'val_macro_f1': metric_value,
                'val_macro_f1_ema': ema_value,
            }
            torch.save(best_state['model_state'], checkpoint_path)
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(
                    f"Early stopping triggered at epoch {epoch}. Best EMA metric={best_ema:.4f}"
                )
                break

    if writer is not None:
        writer.close()

    if best_state is None:
        best_state = {
            'epoch': epoch,
            'model_state': {k: v.cpu() for k, v in model.state_dict().items()},
            'optimizer_state': optimizer.state_dict(),
            'metrics': valid_metrics,
            'train_metrics': train_metrics,
            'preds': preds,
            'targets': targets,
            'val_macro_f1': metric_value,
            'val_macro_f1_ema': ema_value,
        }
        torch.save(best_state['model_state'], checkpoint_path)

    model.load_state_dict(best_state['model_state'])
    best_state.update(
        {
            'run_name': run_name,
            'config': copy.deepcopy(config),
            'history': history,
            'checkpoint_path': checkpoint_path,
            'best_f1': best_metric_value,
        }
    )
    return model, history, best_state



In [ ]:
def run_cross_validation(
    config: Dict,
    n_splits: int = 5,
    shuffle: bool = True,
    random_state: int = SEED,
) -> List[Dict]:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
    fold_results: List[Dict] = []

    print(f"\n>>> Starting {n_splits}-fold CV for {config['run_name']} ({config['rnn_type'].upper()})")
    for fold_idx, (train_idx, valid_idx) in enumerate(skf.split(X_train_np, y_train_idx), start=1):
        fold_config = copy.deepcopy(config)
        fold_config['run_name'] = f"{config['run_name']}_fold{fold_idx}"

        X_tr, y_tr = X_train_np[train_idx], y_train_idx[train_idx]
        X_val, y_val = X_train_np[valid_idx], y_train_idx[valid_idx]

        train_loader = make_dataloader_from_arrays(X_tr, y_tr, batch_size=fold_config['batch_size'], shuffle=True)
        valid_loader = make_dataloader_from_arrays(X_val, y_val, batch_size=fold_config['batch_size'], shuffle=False)

        model, history, best_state = fit_model(
            fold_config,
            train_loader,
            valid_loader,
            run_name=fold_config['run_name'],
            tensorboard=fold_config.get('tensorboard', True),
        )
        best_state['fold'] = fold_idx
        best_state['model'] = model
        best_state['history'] = history
        fold_results.append(best_state)

        metrics = best_state['metrics']
        print(
            f"Fold {fold_idx}/{n_splits} | best F1={metrics['f1']:.4f} | "
            f"Accuracy={metrics['accuracy']:.4f}"
        )

    return fold_results



In [ ]:
def summarize_cv_results(cv_results: List[Dict]) -> pd.DataFrame:
    rows = []
    for res in cv_results:
        cfg = res['config']
        metrics = res['metrics']
        rows.append({
            'run_name': res['run_name'],
            'fold': res['fold'],
            'rnn_type': cfg['rnn_type'],
            'bidirectional': cfg.get('bidirectional', False),
            'hidden_size': cfg['hidden_size'],
            'num_layers': cfg['num_layers'],
            'dropout': cfg['dropout'],
            'f1': metrics['f1'],
            'accuracy': metrics['accuracy'],
            'precision': metrics['precision'],
            'recall': metrics['recall'],
            'checkpoint': str(res['checkpoint_path']),
            'log_dir': str(LOG_DIR / res['run_name']),
        })

    df = pd.DataFrame(rows)
    agg = df.groupby(['rnn_type', 'bidirectional'])[['f1', 'accuracy', 'precision', 'recall']].agg(['mean', 'std'])
    agg.columns = ['_'.join(col) for col in agg.columns]
    agg = agg.reset_index()
    return df, agg



In [ ]:
BASE_CONFIG = {
    'hidden_size': 256,
    'num_layers': 2,
    'dropout': 0.3,
    'lr': 2e-3,
    'weight_decay': 1e-5,
    'epochs': 60,
    'batch_size': 64,
    'valid_size': 0.1,
    'seed': SEED,
    'patience': 8,
    'min_improvement': 5e-4,
    'metric_for_best': 'macro_f1',
    'warmup_epochs': 5,
    'scheduler': 'plateau',
    'scheduler_factor': 0.5,
    'scheduler_patience': 3,
    'scheduler_threshold': 1e-3,
    'scheduler_cooldown': 0,
    'max_grad_norm': 5.0,
    'num_workers': 2,
    'pin_memory': False,
    'ema_alpha': 0.1,
    'tensorboard': True,
    'label_smoothing': 0.05,
    'use_focal_loss': False,
    'use_class_weights': False,
}


def prepare_config(
    name: str,
    rnn_type: str,
    bidirectional: bool,
    overrides: Optional[Dict] = None,
) -> Dict:
    config = copy.deepcopy(BASE_CONFIG)
    config.update({
        'run_name': name,
        'rnn_type': rnn_type,
        'bidirectional': bidirectional,
    })
    if overrides:
        config.update(overrides)
    return config


def run_configurations(section_name: str, configs: List[Dict]) -> List[Dict]:
    results: List[Dict] = []
    for cfg in configs:
        print(
            f"\n=== [{section_name}] Running: {cfg['run_name']} "
            f"({cfg['rnn_type'].upper()} - {'BI' if cfg['bidirectional'] else 'UNI'}) ==="
        )
        result = run_experiment(cfg)
        results.append(result)
        print(
            f"Best validation F1: {result['best_f1']:.4f} at epoch {result['epoch']} | "
            f"Accuracy: {result['metrics']['accuracy']:.4f}"
        )
    return results


def run_experiment(config: Dict) -> Dict:
    run_name = config.get('run_name') or f"{config['rnn_type'].upper()}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    config = copy.deepcopy(config)
    config['run_name'] = run_name

    set_seed(config.get('seed', SEED))

    train_loader, valid_loader, (X_tr, y_tr, X_val, y_val), (train_idx, valid_idx) = create_dataloaders(
        X_train_np,
        y_train_idx,
        valid_size=config.get('valid_size', 0.1),
        batch_size=config['batch_size'],
        seed=config.get('seed', SEED),
        num_workers=config.get('num_workers', 4),
        pin_memory=config.get('pin_memory', True),
    )

    class_counts = np.bincount(y_tr, minlength=NUM_CLASSES)
    criterion = build_criterion(config, class_counts=class_counts)
    if hasattr(criterion, 'to'):
        criterion = criterion.to(DEVICE)

    model, history, best_state = fit_model(
        config,
        train_loader,
        valid_loader,
        run_name=run_name,
        criterion=criterion,
        tensorboard=config.get('tensorboard', True),
    )

    best_state['data_split'] = {
        'X_train': X_tr,
        'y_train': y_tr,
        'X_valid': X_val,
        'y_valid': y_val,
    }
    best_state['split_idx'] = {
        'train_idx': train_idx,
        'valid_idx': valid_idx,
    }
    best_state['feature_names'] = FEATURE_COLUMNS
    best_state['label2idx'] = LABEL2IDX
    best_state['idx2label'] = IDX2LABEL
    best_state['scaler_mean'] = feat_mean
    best_state['scaler_std'] = feat_std
    best_state['model'] = model
    best_state['history'] = history
    return best_state



In [ ]:
# Containers for experiment outputs
rnn_results: List[Dict] = []
gru_results: List[Dict] = []
lstm_results: List[Dict] = []
sweep_results: List[Dict] = []
cv_results: List[Dict] = []
auto_cv_results: List[Dict] = []



In [ ]:
# --- RNN Experiments ---
RNN_CONFIGS = [
    prepare_config('RNN_SINGLE', 'rnn', False),
    prepare_config('RNN_BI', 'rnn', True),
]
rnn_results = run_configurations('RNN', RNN_CONFIGS)

# --- GRU Experiments ---
GRU_CONFIGS = [
    prepare_config('GRU_SINGLE', 'gru', False, {'use_class_weights': True}),
    prepare_config('GRU_BI', 'gru', True, {'use_class_weights': True}),
]
gru_results = run_configurations('GRU', GRU_CONFIGS)

# --- LSTM Experiments ---
LSTM_CONFIGS = [
    prepare_config('LSTM_SINGLE', 'lstm', False, {'dropout': 0.4, 'epochs': 80}),
    prepare_config('LSTM_BI', 'lstm', True, {'dropout': 0.4, 'epochs': 80}),
]
lstm_results = run_configurations('LSTM', LSTM_CONFIGS)



In [ ]:
# (Cross-validation verrà eseguita dopo aver costruito summary_table)


In [ ]:
SWEEP_PARAM_SPACE = {
    'hidden_size': [192, 256, 320],
    'num_layers': [2, 3],
    'dropout': [0.30, 0.45],
    'bidirectional': [True, False],
    'lr': [1e-3, 2e-3],
}
MAX_SWEEP_RUNS = 12
SWEEP_SEED = 42

param_grid = list(product(
    SWEEP_PARAM_SPACE['hidden_size'],
    SWEEP_PARAM_SPACE['num_layers'],
    SWEEP_PARAM_SPACE['dropout'],
    SWEEP_PARAM_SPACE['bidirectional'],
    SWEEP_PARAM_SPACE['lr'],
))
random.Random(SWEEP_SEED).shuffle(param_grid)
param_grid = param_grid[:MAX_SWEEP_RUNS]

sweep_results: List[Dict] = []
for idx, (hidden_size, num_layers, dropout, bidirectional, lr) in enumerate(param_grid, start=1):
    run_name = (
        f"GRU_SWEEP_{idx:02d}_HS{hidden_size}_L{num_layers}_DO{int(dropout*100)}_"
        f"{'BI' if bidirectional else 'UNI'}_LR{lr:.0e}"
    )
    cfg = prepare_config(
        run_name,
        'gru',
        bidirectional,
        overrides={
            'hidden_size': hidden_size,
            'num_layers': num_layers,
            'dropout': dropout,
            'lr': lr,
        },
    )
    print(f"\n>>> Sweep run {idx}/{len(param_grid)} — {run_name}")
    result = run_experiment(cfg)
    sweep_results.append(result)

print(f"\nCompleted {len(sweep_results)} GRU sweep runs.")



In [ ]:
def gather_results(source_names: List[str]) -> List[Dict]:
    collected: List[Dict] = []
    for name in source_names:
        if name in globals():
            value = globals()[name]
            if isinstance(value, list) and value:
                collected.extend(value)
    return collected


def build_summary_table(result_sources: List[str]) -> Tuple[pd.DataFrame, List[Dict]]:
    collected_results = gather_results(result_sources)
    if not collected_results:
        raise RuntimeError('No experiment results available. Run the training/sweep cells first.')

    summary_rows = []
    for res in collected_results:
        cfg = res['config']
        metrics = res['metrics']
        summary_rows.append({
            'run_name': res['run_name'],
            'fold': res.get('fold'),
            'rnn_type': cfg['rnn_type'],
            'bidirectional': cfg.get('bidirectional', False),
            'hidden_size': cfg['hidden_size'],
            'num_layers': cfg['num_layers'],
            'dropout': cfg['dropout'],
            'lr': cfg['lr'],
            'weight_decay': cfg.get('weight_decay', 0.0),
            'use_class_weights': cfg.get('use_class_weights', False),
            'use_focal_loss': cfg.get('use_focal_loss', False),
            'label_smoothing': cfg.get('label_smoothing', 0.0),
            'best_epoch': res['epoch'],
            'best_f1': res['best_f1'],
            'accuracy': metrics['accuracy'],
            'precision': metrics['precision'],
            'recall': metrics['recall'],
            'checkpoint': str(res['checkpoint_path']),
            'log_dir': str(LOG_DIR / res['run_name']),
            'is_cv': res.get('fold') is not None,
        })

    summary_df = pd.DataFrame(summary_rows)
    summary_df = summary_df.sort_values(by='best_f1', ascending=False).reset_index(drop=True)
    return summary_df, collected_results


RESULT_SOURCES = ['rnn_results', 'gru_results', 'lstm_results', 'sweep_results', 'cv_results', 'auto_cv_results']
summary_table, all_results = build_summary_table(RESULT_SOURCES)
summary_table



In [ ]:
for _, row in summary_table.iterrows():
    print(
        f"Run: {row['run_name']} | Model: {row['rnn_type'].upper()} | "
        f"Bidirectional: {row['bidirectional']} | F1: {row['best_f1']:.4f}"
    )
    print(f"  Logs: {row['log_dir']}")

# Per TensorBoard combinato (eseguire su Colab / locale):
# %tensorboard --logdir "{LOG_DIR.as_posix()}"



In [ ]:
# Cross-validation sul modello migliore
if summary_table.empty:
    raise RuntimeError('summary_table is empty; eseguire prima gli esperimenti.')

BEST_CV_SPLITS = 5
best_row = summary_table.iloc[0]
best_cv_cfg = prepare_config(
    f"{best_row['run_name']}_CV",
    best_row['rnn_type'],
    best_row['bidirectional'],
    overrides={
        'hidden_size': best_row['hidden_size'],
        'num_layers': best_row['num_layers'],
        'dropout': best_row['dropout'],
        'lr': best_row['lr'],
        'weight_decay': best_row['weight_decay'],
        'use_class_weights': best_row['use_class_weights'],
        'use_focal_loss': best_row['use_focal_loss'],
        'label_smoothing': best_row['label_smoothing'],
    },
)
cv_results.extend(run_cross_validation(best_cv_cfg, n_splits=BEST_CV_SPLITS))
summary_table, all_results = build_summary_table(RESULT_SOURCES)
cv_folds_df, cv_summary = summarize_cv_results(cv_results)
cv_summary


In [ ]:
ENABLE_AUTO_CV = False
AUTO_CV_TOP_K = 2
AUTO_CV_SPLITS = 5

if ENABLE_AUTO_CV:
    auto_cv_results = []
    for _, row in summary_table.head(AUTO_CV_TOP_K).iterrows():
        cfg = prepare_config(
            f"{row['run_name']}_AUTO_CV",
            row['rnn_type'],
            row['bidirectional'],
            overrides={
                'hidden_size': row['hidden_size'],
                'num_layers': row['num_layers'],
                'dropout': row['dropout'],
                'lr': row['lr'],
                'weight_decay': row['weight_decay'],
                'use_class_weights': row['use_class_weights'],
                'use_focal_loss': row['use_focal_loss'],
                'label_smoothing': row['label_smoothing'],
            },
        )
        cv_out = run_cross_validation(cfg, n_splits=AUTO_CV_SPLITS)
        auto_cv_results.extend(cv_out)

    globals()['auto_cv_results'] = auto_cv_results
    print(f"Completed auto CV for top {min(AUTO_CV_TOP_K, len(summary_table))} runs.")
    RESULT_SOURCES = ['rnn_results', 'gru_results', 'lstm_results', 'sweep_results', 'cv_results', 'auto_cv_results']
    summary_table, all_results = build_summary_table(RESULT_SOURCES)
    summary_table
else:
    print('Auto CV disabled. Set ENABLE_AUTO_CV=True to enable automatic cross-validation.')



In [ ]:
if 'all_results' not in globals() or not all_results:
    raise RuntimeError('No experiment results available. Run the training/sweep cells first.')

best_run = max(all_results, key=lambda x: x['best_f1'])
best_model = best_run['model']
best_history = best_run['history']
print(
    f"Selected best run: {best_run['run_name']} | "
    f"F1={best_run['best_f1']:.4f} | Accuracy={best_run['metrics']['accuracy']:.4f}"
)



In [ ]:
def plot_history(history: Dict[str, List[float]], title: str = 'Learning Curves'):
    epochs = range(1, len(history['train_loss']) + 1)
    fig, axs = plt.subplots(1, 3, figsize=(18, 4))

    axs[0].plot(epochs, history['train_loss'], label='Train')
    axs[0].plot(epochs, history['valid_loss'], label='Valid')
    axs[0].set_title('Loss')
    axs[0].set_xlabel('Epoch')
    axs[0].legend()

    axs[1].plot(epochs, history['train_accuracy'], label='Train')
    axs[1].plot(epochs, history['valid_accuracy'], label='Valid')
    axs[1].set_title('Accuracy')
    axs[1].set_xlabel('Epoch')
    axs[1].legend()

    axs[2].plot(epochs, history['train_f1'], label='Train F1')
    axs[2].plot(epochs, history['valid_f1'], label='Valid F1')
    axs[2].set_title('Macro F1')
    axs[2].set_xlabel('Epoch')
    axs[2].legend()

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


plot_history(best_history, title=f"Learning Curves — {best_run['run_name']}")



In [ ]:
best_preds = best_run['preds']
best_targets = best_run['targets']
print(f"Best validation macro F1: {best_run['best_f1']:.3f}")
print(
    classification_report(
        best_targets,
        best_preds,
        target_names=[IDX2LABEL[i] for i in range(NUM_CLASSES)],
    )
)

cf = confusion_matrix(best_targets, best_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(
    cf,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=[IDX2LABEL[i] for i in range(NUM_CLASSES)],
    yticklabels=[IDX2LABEL[i] for i in range(NUM_CLASSES)],
)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f"Validation Confusion Matrix — {best_run['run_name']}")
plt.show()



In [ ]:
test_loader = DataLoader(TimeSeriesDataset(X_test_np), batch_size=256, shuffle=False)

best_model.eval()
test_preds = []
with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to(DEVICE)
        logits = best_model(inputs)
        test_preds.append(torch.argmax(logits, dim=1).cpu().numpy())

test_preds = np.concatenate(test_preds)
test_labels = [IDX2LABEL[idx] for idx in test_preds]

submission = pd.DataFrame({
    'sample_index': pd.unique(X_test_raw['sample_index']),
    'label': test_labels,
})
submission_filename = OUTPUT_DIR / f"submission_{best_run['run_name'].lower()}.csv"
submission.to_csv(submission_filename, index=False)
print(f"Saved submission to {submission_filename}")
submission.head()



In [ ]:
%tensorboard --logdir "/content/drive/MyDrive/[2025-2026]\ AN2DL/Challenge/outputs/logs"

## Next Steps

- Espandere `EXPERIMENT_CONFIGS` con ricerche random/grid su hidden size, depth, dropout, learning rate e scheduler per automatizzare l'hyperparameter tuning.
- Utilizzare `run_cross_validation` su più configurazioni e confrontare le metriche aggregate in `cv_summary`, esportando i risultati (CSV/LaTeX) per il report finale.
- Monitorare tutti i run con `%tensorboard --logdir outputs/logs`, salvando screenshot delle curve principali e confrontando tempi/risorse.
- Integrare tecniche di regularizzazione avanzate (label smoothing, mixup temporale, stochastic weight averaging) o layer di attention/pooling.
- Costruire ensemble sui checkpoint migliori (media delle probabilità o voting) prima della submission Kaggle definitiva.

